In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from math import sqrt
from sklearn import metrics

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': .3,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.3,
    'learning_rate': 0.03,
    'max_depth': 3,  
    'metric':'auc',
    'min_data_in_leaf': 100,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 100,
    'num_threads': 8,
    'tree_learner': 'serial',
    'is_unbalance': True,
    'objective': 'binary', 
    'verbosity': 1,
}

In [3]:
param = {'bagging_fraction': 0.5166,
   'bagging_freq': 3,
   'lambda_l1': 3.968,
   'lambda_l2': 1.263,
   'learning_rate': 0.0141,
   'max_depth': 3,
   'min_data_in_leaf': 17,
   'min_gain_to_split': 0.2525,
   'min_sum_hessian_in_leaf': 19.55,
   'num_leaves': 20,
   'feature_fraction': 1,
   'save_binary': True,
   'seed': 2319,
   'feature_fraction_seed': 2319,
   'bagging_seed': 2319,
   'drop_seed': 2319,
   'data_random_seed': 2319,
   'objective': 'binary',
   'boosting_type': 'gbdt',
   'verbosity': -1,
   'metric': 'auc',
   'is_unbalance': True,
   'boost_from_average': 'false',
   'num_threads': 6}

In [3]:
ruta = 'Data/train.csv'
df = pd.read_csv(ruta)
df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [24]:
df.shape

(200000, 202)

In [0]:
df.drop_duplicates( inplace=True, keep='first')

In [26]:
df.shape

(200000, 202)

In [4]:
del df['ID_code']
df['target'] =  pd.Categorical(df['target'])
df['target'] =  df['target'].cat.codes
Y = df['target'].values
del df['target']
X = df.values

In [5]:
features = df.columns
features


Index(['var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7',
       'var_8', 'var_9',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=200)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [7]:
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve

folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=44000)

cv_real = []
cv_pred = []
cv_thresholds = []

feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, Y)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(X[trn_idx], label=Y[trn_idx])
    val_data = lgb.Dataset(X[val_idx], label=Y[val_idx])




    num_round = 1000000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data,val_data], verbose_eval=1000, early_stopping_rounds = 1000)
    
    
    ###########################################################################################
    
    y_pred = clf.predict(X[val_idx], num_iteration=clf.best_iteration)
    print(y_pred)
    cv_real.extend(Y[val_idx])
    cv_pred.extend(y_pred)

    cm = confusion_matrix(Y[val_idx], y_pred>0.1)
    print ("Confusion Matrix")
    print (cm)

    acc = metrics.accuracy_score(Y[val_idx],  y_pred>0.1)
    precision = metrics.precision_score(Y[val_idx],  y_pred>0.1, average='macro')

    print("acc: ",acc )
    print("precision", precision)

    print("Obtener el threshold correcto ...")
    false_positive_rate, true_positive_rate, thresholds_ = metrics.roc_curve(Y[val_idx], y_pred)
    auc = metrics.auc(false_positive_rate, true_positive_rate)
    print("roc_curve: ", auc)
    roc_auc_score = metrics.roc_auc_score(Y[val_idx], y_pred)
    print("roc_auc_score: ", roc_auc_score)

    threshold = Find_Optimal_Cutoff(Y[val_idx], y_pred)
    print("threshold: ", threshold[0])
    cv_thresholds.extend(threshold)


    print("Aplicar Obtener el threshold correcto ...")
    y_pred = (clf.predict(X[val_idx], num_iteration=clf.best_iteration) > threshold[0])
    cm = confusion_matrix(Y[val_idx], y_pred)
    print ("Confusion Matrix")
    print (cm)

    false_positive_rate, true_positive_rate, thresholds_ = metrics.roc_curve(Y[val_idx], y_pred)
    auc = metrics.auc(false_positive_rate, true_positive_rate)
  

    roc_auc_score = metrics.roc_auc_score(Y[val_idx], y_pred)
    print("roc_auc_score: ", roc_auc_score)


    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    




Fold 0
Training until validation scores don't improve for 1000 rounds
[1000]	training's auc: 0.906034	valid_1's auc: 0.879003
[2000]	training's auc: 0.925582	valid_1's auc: 0.891883
[3000]	training's auc: 0.934914	valid_1's auc: 0.895853
[4000]	training's auc: 0.942482	valid_1's auc: 0.896015
Early stopping, best iteration is:
[3661]	training's auc: 0.939999	valid_1's auc: 0.89649
[0.08240265 0.80294865 0.04433571 ... 0.92817002 0.8222961  0.69179565]
Confusion Matrix
[[ 6256 11735]
 [   39  1971]]
acc:  0.4113294335283236
precision 0.5688051197002478
Obtener el threshold correcto ...
roc_curve:  0.8964903125968734
roc_auc_score:  0.8964903125968734
threshold:  0.43071525498624286
Aplicar Obtener el threshold correcto ...
Confusion Matrix
[[14660  3331]
 [  372  1638]]
roc_auc_score:  0.8148886217569814
Fold 1
Training until validation scores don't improve for 1000 rounds
[1000]	training's auc: 0.905748	valid_1's auc: 0.879798
[2000]	training's auc: 0.925607	valid_1's auc: 0.89217
[300

In [18]:
metrics.roc_auc_score(cv_real,cv_pred)

0.8973690628961856

In [40]:
false_positive_rate, true_positive_rate, thresholds_ = metrics.roc_curve(Y,oof)
auc = metrics.auc(false_positive_rate, true_positive_rate)
print("roc_curve: ", auc)
roc_auc_score = metrics.roc_auc_score(Y, oof)
print("roc_auc_score: ", roc_auc_score)


cm = confusion_matrix(Y, oof)
print ("Confusion Matrix")
print (cm)

roc_curve:  0.6292373833402928
roc_auc_score:  0.6292373833402928
Confusion Matrix
[[178880   1022]
 [ 14789   5309]]


In [0]:
tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()

In [42]:
 confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0])

array([[0, 2],
       [1, 1]])

In [43]:
tn, fp, fn, tp

(0, 2, 1, 1)

In [29]:
X

array([[ 1.97587719, -0.58607933, -2.52157124, ...,  0.47015451,
         1.56094515,  0.99612985],
       [ 0.89603199, -0.74528385, -0.30561523, ..., -1.09668324,
        -1.00561084,  1.71384907],
       [-0.34363376,  0.02901586, -0.0251468 , ...,  1.32079613,
        -0.81297923,  0.17929989],
       ...,
       [-1.40784668, -0.97644548, -1.69231018, ..., -0.96385166,
        -0.55447149,  0.24070977],
       [-0.67620196, -0.67179368,  0.37963812, ...,  0.08583192,
         0.3150405 ,  0.00970278],
       [ 0.19677662,  0.4212418 , -0.35077952, ..., -1.1047794 ,
         0.07046183,  0.72320794]])

In [32]:
45*6

270

NameError: name 'Sequential' is not defined